# 2.3 Chained Tool Calls

Not all tool calls are independent. Sometimes the output of one tool feeds into the next: search for users, then get details for the one you found, then look up their orders.

This is multi-turn tool use. The model calls a tool, sees the result, reasons about what to do next, and calls another tool. The conversation grows with each round-trip:

```
user -> assistant(tool_call_1) -> tool(result_1) -> assistant(tool_call_2) -> tool(result_2) -> assistant(answer)
```

The model drives the chain. You don't hardcode the sequence -- the model decides what to call next based on what it learned from the previous result.

In [1]:
import os
import json
import openai
from dotenv import load_dotenv

load_dotenv()

client = openai.OpenAI(
    api_key=os.getenv('OPENROUTER_API_KEY'),
    base_url='https://openrouter.ai/api/v1',
)

MODEL = 'google/gemini-2.5-flash-lite'

## Define chained tools

Three tools that form a dependency chain:

1. `search_users(query)` -- find users matching a search query
2. `get_user_profile(user_id)` -- get full profile for a specific user
3. `get_user_orders(user_id)` -- get order history for a specific user

The model has to search first (to find the user ID), then use that ID to get profile or order details. It can't skip ahead.

In [2]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "search_users",
            "description": "Search for users by name or email. Returns a list of matching user IDs and names.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query (name or email)"},
                },
                "required": ["query"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_user_profile",
            "description": "Get detailed profile information for a user by their ID.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {"type": "string", "description": "The user ID"},
                },
                "required": ["user_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_user_orders",
            "description": "Get order history for a user by their ID. Returns list of recent orders.",
            "parameters": {
                "type": "object",
                "properties": {
                    "user_id": {"type": "string", "description": "The user ID"},
                },
                "required": ["user_id"],
            },
        },
    },
]

# Simulated database
USERS_DB = {
    "u_101": {"name": "Alice Chen", "email": "alice@example.com", "plan": "Premium", "joined": "2023-01-15"},
    "u_102": {"name": "Bob Smith", "email": "bob@example.com", "plan": "Basic", "joined": "2023-06-20"},
    "u_103": {"name": "Carol Davis", "email": "carol@example.com", "plan": "Premium", "joined": "2024-02-01"},
}

ORDERS_DB = {
    "u_101": [
        {"order_id": "ORD-001", "item": "Wireless Headphones", "amount": 89.99, "status": "Delivered"},
        {"order_id": "ORD-004", "item": "USB-C Hub", "amount": 45.00, "status": "Shipped"},
    ],
    "u_102": [
        {"order_id": "ORD-002", "item": "Keyboard", "amount": 129.99, "status": "Delivered"},
    ],
    "u_103": [
        {"order_id": "ORD-003", "item": "Monitor Stand", "amount": 59.99, "status": "Processing"},
        {"order_id": "ORD-005", "item": "Webcam", "amount": 79.99, "status": "Delivered"},
        {"order_id": "ORD-006", "item": "Desk Lamp", "amount": 34.99, "status": "Shipped"},
    ],
}


def search_users(query: str) -> dict:
    """Search users by name or email."""
    query_lower = query.lower()
    matches = []
    for uid, info in USERS_DB.items():
        if query_lower in info['name'].lower() or query_lower in info['email'].lower():
            matches.append({'user_id': uid, 'name': info['name'], 'email': info['email']})
    return {"results": matches, "total": len(matches)}


def get_user_profile(user_id: str) -> dict:
    """Get full profile for a user."""
    user = USERS_DB.get(user_id)
    if not user:
        return {"error": f"User {user_id} not found"}
    return {"user_id": user_id, **user}


def get_user_orders(user_id: str) -> dict:
    """Get order history for a user."""
    orders = ORDERS_DB.get(user_id, [])
    total_spent = sum(o['amount'] for o in orders)
    return {"user_id": user_id, "orders": orders, "total_spent": total_spent}


available_functions = {
    "search_users": search_users,
    "get_user_profile": get_user_profile,
    "get_user_orders": get_user_orders,
}

print(f'Defined {len(tools)} chained tools')
print(f'Users in DB: {list(USERS_DB.keys())}')

Defined 3 chained tools
Users in DB: ['u_101', 'u_102', 'u_103']


## The multi-turn tool loop

Unlike 2.1 (one round-trip) or 2.2 (one round-trip with parallel calls), chained tools require multiple round-trips. The model calls a tool, sees the result, then decides whether to call another tool or give the final answer.

We loop until the model stops requesting tool calls.

In [3]:
SYSTEM_PROMPT = """You are a helpful assistant with access to a user database.
When a user asks about a person, ALWAYS use the available tools to find the answer.
Do not ask clarifying questions -- use search_users to find the user, then use
get_user_profile or get_user_orders to get the details they asked about.
Always follow through until you have the complete answer."""


def run_chained(user_message: str, max_rounds: int = 5) -> str:
    """
    Run a multi-turn tool-calling conversation.

    The model can call tools across multiple rounds. Each round:
    1. Model sees the full conversation history
    2. Model either calls tool(s) or gives a final answer
    3. If tool(s) called, execute them and add results to history
    4. Loop until model gives a text answer or max_rounds reached
    """
    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": user_message},
    ]

    print(f'User: {user_message}')
    print()

    for round_num in range(1, max_rounds + 1):
        response = client.chat.completions.create(
            model=MODEL, messages=messages, tools=tools,
        )

        assistant_msg = response.choices[0].message

        # If no tool calls, we have the final answer
        if not assistant_msg.tool_calls:
            print(f'--- Round {round_num}: Final answer ---')
            print(f'{assistant_msg.content}')
            return assistant_msg.content

        # Process tool calls for this round
        print(f'--- Round {round_num}: {len(assistant_msg.tool_calls)} tool call(s) ---')
        messages.append(assistant_msg)

        for tc in assistant_msg.tool_calls:
            fn_name = tc.function.name
            fn_args = json.loads(tc.function.arguments)
            print(f'  Call: {fn_name}({json.dumps(fn_args)})')

            # Execute
            fn = available_functions[fn_name]
            result = fn(**fn_args)
            print(f'  Result: {json.dumps(result)}')

            messages.append({
                "role": "tool",
                "tool_call_id": tc.id,
                "content": json.dumps(result),
            })
        print()

    return 'Max rounds reached without final answer'


## Run it: chained queries

In [4]:
# Test 1: Requires search -> profile (2 rounds)
print('=== Test 1: Search then get profile ===')
print()
run_chained("Can you find Alice's account and tell me what plan she's on?")

print()
print('=' * 60)
print()

# Test 2: Requires search -> orders (2 rounds)
print('=== Test 2: Search then get orders ===')
print()
run_chained("How much has Carol Davis spent on orders?")

print()
print('=' * 60)
print()

# Test 3: Requires search -> profile + orders (2-3 rounds, may use parallel)
print('=== Test 3: Search then get both profile and orders ===')
print()
run_chained("Find Bob Smith and give me his full profile and order history.")

=== Test 1: Search then get profile ===

User: Can you find Alice's account and tell me what plan she's on?



--- Round 1: 1 tool call(s) ---
  Call: search_users({"query": "Alice"})
  Result: {"results": [{"user_id": "u_101", "name": "Alice Chen", "email": "alice@example.com"}], "total": 1}



--- Round 2: 1 tool call(s) ---
  Call: get_user_profile({"user_id": "u_101"})
  Result: {"user_id": "u_101", "name": "Alice Chen", "email": "alice@example.com", "plan": "Premium", "joined": "2023-01-15"}



--- Round 3: Final answer ---
Alice is on the Premium plan.


=== Test 2: Search then get orders ===

User: How much has Carol Davis spent on orders?



--- Round 1: 1 tool call(s) ---
  Call: search_users({"query": "Carol Davis"})
  Result: {"results": [{"user_id": "u_103", "name": "Carol Davis", "email": "carol@example.com"}], "total": 1}



...

--- Round 3: Final answer ---
Carol Davis has spent $174.97 on orders.


=== Test 3: Search then get both profile and orders ===

User: Find Bob Smith and give me his full profile and order history.



--- Round 1: 1 tool call(s) ---
  Call: search_users({"query": "Bob Smith"})
  Result: {"results": [{"user_id": "u_102", "name": "Bob Smith", "email": "bob@example.com"}], "total": 1}



--- Round 2: 2 tool call(s) ---
  Call: get_user_profile({"user_id": "u_102"})
  Result: {"user_id": "u_102", "name": "Bob Smith", "email": "bob@example.com", "plan": "Basic", "joined": "2023-06-20"}
  Call: get_user_orders({"user_id": "u_102"})
  Result: {"user_id": "u_102", "orders": [{"order_id": "ORD-002", "item": "Keyboard", "amount": 129.99, "status": "Delivered"}], "total_spent": 129.99}



--- Round 3: Final answer ---
Bob Smith (u_102) has the Basic plan and joined on 2023-06-20. His total spending is $129.99. His recent order is a Keyboard for $129.99, which has been delivered.


'Bob Smith (u_102) has the Basic plan and joined on 2023-06-20. His total spending is $129.99. His recent order is a Keyboard for $129.99, which has been delivered.'

## Conversation growth

Each round adds messages to the conversation. Let's visualize how the message history grows through a chained interaction.

In [5]:
# Trace the conversation structure for a chained query
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user", "content": "What plan is Alice on and what are her recent orders?"},
]

print('Conversation trace:')
print('=' * 60)
print(f'[Start] 2 messages (system + user)')

for round_num in range(1, 6):
    response = client.chat.completions.create(
        model=MODEL, messages=messages, tools=tools,
    )
    assistant_msg = response.choices[0].message

    if not assistant_msg.tool_calls:
        messages.append({'role': 'assistant', 'content': assistant_msg.content})
        print(f'[Round {round_num}] +1 assistant message (final answer)')
        print(f'  Total messages: {len(messages)}')
        break

    messages.append(assistant_msg)
    n_calls = len(assistant_msg.tool_calls)
    tool_names = [tc.function.name for tc in assistant_msg.tool_calls]
    print(f'[Round {round_num}] +1 assistant message ({n_calls} tool call(s): {tool_names})')

    for tc in assistant_msg.tool_calls:
        fn = available_functions[tc.function.name]
        args = json.loads(tc.function.arguments)
        result = fn(**args)
        messages.append({'role': 'tool', 'tool_call_id': tc.id, 'content': json.dumps(result)})
    print(f'  +{n_calls} tool result(s)')
    print(f'  Total messages: {len(messages)}')

print()
print(f'Final conversation: {len(messages)} messages')
for i, msg in enumerate(messages):
    if isinstance(msg, dict):
        role = msg['role']
        content = msg.get('content', '')[:80]
    else:
        role = msg.role
        content = ''
        if msg.tool_calls:
            content = f'tool_calls: {[tc.function.name for tc in msg.tool_calls]}'
        elif msg.content:
            content = msg.content[:80]
    print(f'  [{i}] {role}: {content}')


Conversation trace:
[Start] 2 messages (system + user)


[Round 1] +1 assistant message (1 tool call(s): ['search_users'])
  +1 tool result(s)
  Total messages: 4


[Round 2] +1 assistant message (2 tool call(s): ['get_user_profile', 'get_user_orders'])
  +2 tool result(s)
  Total messages: 7


[Round 3] +1 assistant message (final answer)
  Total messages: 8

Final conversation: 8 messages
  [0] system: You are a helpful assistant with access to a user database.
When a user asks abo
  [1] user: What plan is Alice on and what are her recent orders?
  [2] assistant: tool_calls: ['search_users']
  [3] tool: {"results": [{"user_id": "u_101", "name": "Alice Chen", "email": "alice@example.
  [4] assistant: tool_calls: ['get_user_profile', 'get_user_orders']
  [5] tool: {"user_id": "u_101", "name": "Alice Chen", "email": "alice@example.com", "plan":
  [6] tool: {"user_id": "u_101", "orders": [{"order_id": "ORD-001", "item": "Wireless Headph
  [7] assistant: 
